In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import json,os
%matplotlib inline

In [2]:
df_reviews = pd.read_csv('Yelp_dataset/review_json_to_pd_to_csv.csv')
df_business = pd.read_csv('Yelp_dataset/business_json_to_pd_to_csv.csv')
df_users = pd.read_csv('Yelp_dataset/user_json_to_pd_to_csv.csv')

In [3]:
temp = pd.merge(df_business,df_reviews,on='business_id',how='inner')

In [4]:
temp.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id
0,"4855 E Warner Rd, Ste B9","{u'AcceptsInsurance': True, u'ByAppointmentOnl...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{u'Tuesday': u'7:30-17:00', u'Friday': u'7:30-...",1,33.33069,-111.978599,Dental by Design,...,4.0,AZ,0,2017-02-22,0.0,3_-fne2m2GvKESZRlULAiw,5.0,"They make a plan, for your mouth to get health...",0.0,jTvKkNTat9QftSxL_FMcQw
1,"4855 E Warner Rd, Ste B9","{u'AcceptsInsurance': True, u'ByAppointmentOnl...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{u'Tuesday': u'7:30-17:00', u'Friday': u'7:30-...",1,33.33069,-111.978599,Dental by Design,...,4.0,AZ,0,2015-01-20,0.0,5gJk64TDq5lqSuoNiaqjVQ,5.0,I have extreme anxiety when it comes to the de...,1.0,IG6XMgQyWF2_RW1nKEioxA


In [5]:
df_merged = pd.merge(temp,df_users,on='user_id',how='inner')

In [6]:
df_merged.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name_x,...,compliment_writer,cool_y,elite,fans,friends,funny_y,name_y,review_count_y,useful_y,yelping_since
0,"4855 E Warner Rd, Ste B9","{u'AcceptsInsurance': True, u'ByAppointmentOnl...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{u'Tuesday': u'7:30-17:00', u'Friday': u'7:30-...",1,33.330690,-111.978599,Dental by Design,...,0,31,[],1,[20NU0t5ByjoHnPWTtckN1g],38,Linz,25,174,2011-06-19
1,4252 S 48th St,"{u'GoodForMeal': {u'dessert': False, u'latenig...",g-aAe8SqZlZTnN3oA8mNlw,"[Mexican, Restaurants, Breakfast & Brunch, Foo...",Phoenix,"{u'Monday': u'6:00-0:00', u'Tuesday': u'6:00-0...",1,33.407691,-111.978612,Del Taco,...,0,31,[],1,[20NU0t5ByjoHnPWTtckN1g],38,Linz,25,174,2011-06-19


In [7]:
df_final = df_merged[(df_merged.categories.str.contains("Restaurants|Food",case=False)) & (df_merged.city == "Las Vegas")][['business_id', 'name_x', 'review_count_x', 'stars_x', 'user_id', 'name_y', 'review_count_y', 'stars_y', 'city', 'average_stars','review_count_y', 'review_id']]

In [8]:
df_final.head(2)

,business_id,name_x,review_count_x,stars_x,user_id,name_y,review_count_y,stars_y,city,average_stars,review_count_y,review_id
87,DVJkxZkLKS76Khb2y6XA6A,Protein House,382,4.0,YcmNpPM0ag94g4T0zAtdcg,Jenn,55,4.0,Las Vegas,4.02,55,mPq_g2Cx_pMQ72_-BGRCVg
594,iCQpiavjjPzJ5_3gPD5Ebg,Secret Pizza,3741,4.0,NnrCwpS1i5mZtkGGzmIk0g,Serena,428,3.0,Las Vegas,3.80,428,fGtEJo9Rf-EpMBvHYjiXLw


In [9]:
cols = df_final.columns.values
cols[1] = 'business_name'
cols[2] = 'business_review_count'
cols[3] = 'business_average'
cols[5] = 'user_name'
cols[6] = 'user_review_count'
cols[7] = 'stars'
cols[9] = 'user_avg'
df_final.columns = cols
df_final.head(2)

,business_id,business_name,business_review_count,business_average,user_id,user_name,user_review_count,stars,city,user_avg,review_count_y,review_id
87,DVJkxZkLKS76Khb2y6XA6A,Protein House,382,4.0,YcmNpPM0ag94g4T0zAtdcg,Jenn,55,4.0,Las Vegas,4.02,55,mPq_g2Cx_pMQ72_-BGRCVg
594,iCQpiavjjPzJ5_3gPD5Ebg,Secret Pizza,3741,4.0,NnrCwpS1i5mZtkGGzmIk0g,Serena,428,3.0,Las Vegas,3.80,428,fGtEJo9Rf-EpMBvHYjiXLw


In [10]:
df_final=df_final.drop('review_count_y',axis=1)

In [11]:
df_final.shape

(1003628, 11)

In [12]:
df_reduced = df_final[(df_final.user_review_count >50)&(df_final.business_review_count > 100)]

In [13]:
df_reduced.head(2)

,business_id,business_name,business_review_count,business_average,user_id,user_name,user_review_count,stars,city,user_avg,review_id
87,DVJkxZkLKS76Khb2y6XA6A,Protein House,382,4.0,YcmNpPM0ag94g4T0zAtdcg,Jenn,55,4.0,Las Vegas,4.02,mPq_g2Cx_pMQ72_-BGRCVg
594,iCQpiavjjPzJ5_3gPD5Ebg,Secret Pizza,3741,4.0,NnrCwpS1i5mZtkGGzmIk0g,Serena,428,3.0,Las Vegas,3.80,fGtEJo9Rf-EpMBvHYjiXLw


In [14]:
df_ratings = df_reduced[['user_id','business_id','stars']].groupby(['user_id','business_id']).head(1)

In [15]:
from surprise import Dataset
from surprise import Reader

In [16]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df_ratings[['user_id','business_id','stars']],reader)

In [17]:
from surprise import SVD
from surprise import SVDpp
from surprise import accuracy
from surprise import NMF
from surprise.model_selection import train_test_split

In [18]:
trainset, testset = train_test_split(data, test_size=.1)
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0528


1.0528143187886523

In [19]:
from surprise import BaselineOnly
from surprise import NormalPredictor

In [20]:
algo = BaselineOnly()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.0345


1.0345455441793632

In [21]:
algo = NormalPredictor()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.5184


1.5183736676968749

In [22]:
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline

In [23]:
algo = KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0894


1.089371086552354

In [24]:
algo = KNNWithMeans()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1382


1.1381685229070941

In [25]:
algo = KNNBaseline()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0528


1.0527607150351956

In [26]:
from surprise.model_selection import cross_validate

In [27]:
cross_validate(BaselineOnly(),data,measures=['RMSE', 'MAE'],cv=5,verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8152  0.8162  0.8177  0.8144  0.8210  0.8169  0.0023  
RMSE (testset)    1.0313  1.0334  1.0379  1.0313  1.0395  1.0347  0.0034  
Fit time          0.60    0.54    0.57    0.61    0.52    0.57    0.04    
Test time         1.11    1.15    1.12    1.15    1.11    1.13    0.02    


{u'fit_time': (0.6016621589660645,
  0.5364909172058105,
  0.5715670585632324,
  0.6141421794891357,
  0.5234370231628418),
 u'test_mae': array([0.81515818, 0.81616361, 0.81772921, 0.81435181, 0.820999  ]),
 u'test_rmse': array([1.03131017, 1.0333934 , 1.03786384, 1.03132263, 1.03950037]),
 u'test_time': (1.1133849620819092,
  1.1487679481506348,
  1.116145133972168,
  1.1503088474273682,
  1.1122279167175293)}

In [28]:
cross_validate(SVD(),data,measures=['RMSE', 'MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8336  0.8277  0.8261  0.8310  0.8271  0.8291  0.0028  
RMSE (testset)    1.0575  1.0504  1.0475  1.0538  1.0481  1.0515  0.0038  
Fit time          20.65   20.88   20.74   20.27   20.33   20.57   0.24    
Test time         1.31    1.31    1.29    1.26    1.27    1.29    0.02    


{u'fit_time': (20.6472008228302,
  20.883193016052246,
  20.74226403236389,
  20.267313957214355,
  20.330577850341797),
 u'test_mae': array([0.83358662, 0.82768925, 0.82606331, 0.83100408, 0.82714189]),
 u'test_rmse': array([1.05753341, 1.05037788, 1.04749987, 1.05379482, 1.04811791]),
 u'test_time': (1.314255952835083,
  1.3090620040893555,
  1.290571928024292,
  1.2588310241699219,
  1.2672338485717773)}